# Azure Open AI and functions

In [1]:
# %pip install --upgrade openai

In [2]:
import openai
import json
import os
import pytz
import inspect
import requests
import folium

from datetime import datetime
from dotenv import load_dotenv
from pprint import pprint

In [3]:
load_dotenv("azure.env")

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

In [4]:
openai.api_version = "2023-07-01-preview"
openai.api_type = "azure"
deployment_name = "gpt-35-turbo-0613"  # Should be the 613

In [5]:
def get_today(location):
    """
    Get current datetime from a location
    """
    try:
        now = datetime.now(pytz.timezone(location))
        today = now.strftime("%d-%b-%Y %H:%M:%S")

        return today
    except:
        return "Error"

In [6]:
get_today("America/New_York")

'06-Sep-2023 05:49:34'

In [7]:
get_today("Europe/Paris")

'06-Sep-2023 11:49:34'

In [8]:
get_today("Europe/London")

'06-Sep-2023 10:49:34'

In [9]:
get_today("Asia/Tokyo")

'06-Sep-2023 18:49:34'

In [10]:
def get_coordinates(location):
    """
    Get coordinates lattitude and longitude using an url
    """
    baseurl = "https://api-adresse.data.gouv.fr/search/"
    params = {"q": location, "limit": 1}
    response = requests.get(baseurl, params=params)

    if response.status_code == 200:
        data = response.json()
        res = data["features"][0]
        long, lat = res["geometry"]["coordinates"]
        return str(long) + ", " + str(lat)
    else:
        print("Error")

In [11]:
get_coordinates("Paris")

'2.347, 48.859'

In [12]:
get_coordinates("London")

'1.192566, 45.858449'

In [13]:
functions = [
    {
        "name": "get_today",
        "description": "Get the current date time in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                }
            },
            "required": ["location"],
        },
    },
    {
        "name": "get_coordinates",
        "description": "Get the coordinates lattitude and longitude from a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Get the coordinates from a location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                }
            },
            "required": ["location"],
        },
    },
]


available_functions = {
    "get_today": get_today,
    "get_coordinates": get_coordinates,
}

In [14]:
pprint(available_functions)

{'get_coordinates': <function get_coordinates at 0x7f48df417520>,
 'get_today': <function get_today at 0x7f48a7efe050>}


In [15]:
# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [16]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message["function_call"]["name"]

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "function_call": {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"],
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages, deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

## Testing

In [17]:
messages = [{"role": "user", "content": "What time is it in New York?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

Output of function call:
06-Sep-2023 05:49:36

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "America/New_York"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 05:49:36'}

{
  "role": "assistant",
  "content": "As of now, the current time in New York is 05:49 AM."
}


In [18]:
print(assistant_response["choices"][0]["message"]["content"])

As of now, the current time in New York is 05:49 AM.


In [19]:
messages = [{"role": "user", "content": "What the current day in Paris?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 11:49:38

Messages in second request:
{'role': 'user', 'content': 'What the current day in Paris?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 11:49:38'}

{
  "role": "assistant",
  "content": "Today is September 6th, 2023 in Paris, France."
}


In [20]:
print(assistant_response["choices"][0]["message"]["content"])

Today is September 6th, 2023 in Paris, France.


In [21]:
messages = [{"role": "user", "content": "What the current date and time in Paris?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n\"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 11:49:41

Messages in second request:
{'role': 'user', 'content': 'What the current date and time in Paris?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n"location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 11:49:41'}

{
  "role": "assistant",
  "content": "The current date and time in Paris is 6th September 2023, 11:49 AM."
}


In [22]:
print(assistant_response["choices"][0]["message"]["content"])

The current date and time in Paris is 6th September 2023, 11:49 AM.


In [23]:
messages = [
    {
        "role": "user",
        "content": "What the current date and time in NYC? Format the time with seconds",
    }
]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

Output of function call:
06-Sep-2023 05:49:43

Messages in second request:
{'role': 'user', 'content': 'What the current date and time in NYC? Format the time with seconds'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "America/New_York"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 05:49:43'}

{
  "role": "assistant",
  "content": "The current date and time in NYC is September 6, 2023, 05:49:43 (formatted with seconds)."
}


In [24]:
print(assistant_response["choices"][0]["message"]["content"])

The current date and time in NYC is September 6, 2023, 05:49:43 (formatted with seconds).


In [25]:
messages = [
    {
        "role": "user",
        "content": "I am in Paris. I will drive for 3 hours. What is my arrival time?",
    }
]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 11:49:46

Messages in second request:
{'role': 'user', 'content': 'I am in Paris. I will drive for 3 hours. What is my arrival time?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 11:49:46'}

{
  "role": "assistant",
  "content": "If you will be driving for 3 hours from the current time, your estimated arrival time would be approximately 3 hours from now."
}


In [26]:
print(assistant_response["choices"][0]["message"]["content"])

If you will be driving for 3 hours from the current time, your estimated arrival time would be approximately 3 hours from now.


In [27]:
messages = [{"role": "user", "content": "I am in Paris. What was the yesterday date?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 11:49:48

Messages in second request:
{'role': 'user', 'content': 'I am in Paris. What was the yesterday date?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 11:49:48'}

{
  "role": "assistant",
  "content": "Yesterday's date in Paris was September 5, 2023."
}


In [28]:
print(assistant_response["choices"][0]["message"]["content"])

Yesterday's date in Paris was September 5, 2023.


In [29]:
messages = [
    {
        "role": "user",
        "content": "I am in Paris. What is the number of days starting from the 1st of January 2023",
    }
]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 11:49:50

Messages in second request:
{'role': 'user', 'content': 'I am in Paris. What is the number of days starting from the 1st of January 2023'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 11:49:50'}

{
  "role": "assistant",
  "content": "If today is September 6, 2023, and you are asking for the number of days starting from the 1st of January 2023, you can calculate it by subtracting the two dates:\n\nSeptember 6, 2023 - January 1, 2023 = 248 days\n\nTherefore, there are 248 days from the 1st of January 2023 to September 6, 2023."
}


In [30]:
print(assistant_response["choices"][0]["message"]["content"])

If today is September 6, 2023, and you are asking for the number of days starting from the 1st of January 2023, you can calculate it by subtracting the two dates:

September 6, 2023 - January 1, 2023 = 248 days

Therefore, there are 248 days from the 1st of January 2023 to September 6, 2023.


In [31]:
messages = [
    {
        "role": "user",
        "content": "I am in Brighton. What is the longitude and lattitude?",
    }
]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_coordinates",
  "arguments": "{\n\"location\": \"Brighton\"\n}"
}

Output of function call:
1.517171, 50.194893

Messages in second request:
{'role': 'user', 'content': 'I am in Brighton. What is the longitude and lattitude?'}
{'role': 'assistant', 'function_call': {'name': 'get_coordinates', 'arguments': '{\n"location": "Brighton"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_coordinates', 'content': '1.517171, 50.194893'}

{
  "role": "assistant",
  "content": "The latitude of Brighton is 50.194893 and the longitude is 1.517171."
}


In [32]:
print(assistant_response["choices"][0]["message"]["content"])

The latitude of Brighton is 50.194893 and the longitude is 1.517171.


In [68]:
messages = [
    {
        "role": "user",
        "content": "I am in Nice. Display the latitude and longitude",
    }
]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_coordinates",
  "arguments": "{\n  \"location\": \"Nice\"\n}"
}

Output of function call:
7.250987, 43.713012

Messages in second request:
{'role': 'user', 'content': 'I am in Nice. Display the latitude and longitude'}
{'role': 'assistant', 'function_call': {'name': 'get_coordinates', 'arguments': '{\n  "location": "Nice"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_coordinates', 'content': '7.250987, 43.713012'}

{
  "role": "assistant",
  "content": "The latitude of Nice is 43.713012 and the longitude is 7.250987."
}


In [69]:
lat = "43.713012"
long = "7.250987"

In [70]:
mymap = folium.Map(location=[lat, long], zoom_start=12)
folium.Marker([lat, long], tooltip="Here I'm am").add_to(mymap)

mymap